In [4]:
import asyncio
from langgraph.graph import MessageGraph, END
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from decouple import config
from openai import OpenAI
import httpx
import json

from typing import List

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage

In [5]:
document = """
Steps 
Step One: We admitted we were powerless over our addiction—that our lives had become unmanageable.
Step Two: Came to believe that a Power greater than ourselves could restore us to sanity.
Step Three: Made a decision to turn our will and our lives over to the care of a higher power as we understood Him.
Step Four: Made a searching and fearless moral inventory of ourselves.
Step Five: Admitted to a higher power, to ourselves, and to another human being the exact nature of our wrongs.
Step Six: Were entirely ready to have a higher power remove all these defects of character.
Step Seven: Humbly asked Him to remove our shortcomings.
Step Eight: Made a list of all persons we had harmed, and became willing to make amends to them all.
Step Nine: Made direct amends to such people wherever possible, except when to do so would injure them or others.
Step Ten: Continued to take personal inventory and when we were wrong promptly admitted it.
Step Eleven: Sought through prayer and meditation to improve our conscious contact with a higher power as we understood Him, praying only for knowledge of His will for us and the power to carry that out.
Step Twelve: Having had a spiritual awakening as the result of these steps, we tried to carry this message to addicts, and to practice these principles in all our affairs.

Traditions
Tradition One: Our common welfare should come first; personal recovery depends upon unity.
Tradition Two: For our group purpose, there is but one ultimate authority—a loving God as He may express Himself in our group conscience. Our leaders are but trusted servants; they do not govern.
Tradition Three: The only requirement for membership is a desire to stop the addictive behavior.
Tradition Four: Each group should be autonomous except in matters affecting other groups or the entire fellowship.
Tradition Five: Each group has but one primary purpose—to carry its message to the addict who still suffers.
Tradition Six: A group ought never endorse, finance, or lend the group name to any related facility or outside enterprise, lest problems of money, property, and prestige divert us from our primary purpose.
Tradition Seven: Every group ought to be fully self-supporting, declining outside contributions.
Tradition Eight: The group should remain forever nonprofessional, but our service centers may employ special workers.
Tradition Nine: The group as such ought never be organized, but we may create service boards or committees directly responsible to those they serve.
Tradition Ten: The group has no opinion on outside issues; hence the group name ought never be drawn into public controversy.
Tradition Eleven: Our public relations policy is based on attraction rather than promotion; we need always maintain personal anonymity at the level of press, radio, and films.
Tradition Twelve: Anonymity is the spiritual foundation of all our traditions, ever reminding us to place principles before personalities.




Promises
Promise One: We are going to know a new freedom and a new happiness.
Promise Two: We will not regret the past nor wish to shut the door on it.
Promise Three: We will comprehend the word serenity.
Promise Four: We will know peace.
Promise Five: No matter how far down the scale we have gone, we will see how our experience can benefit others.
Promise Six: That feeling of uselessness and self-pity will disappear.
Promise Seven: We will lose interest in selfish things and gain interest in our fellows.
Promise Eight: Self-seeking will slip away.
Promise Nine: Our whole attitude and outlook upon life will change.
Promise Ten: Fear of people and economic insecurity will leave us.
Promise Eleven: We will intuitively know how to handle situations which used to baffle us.
Promise Twelve: We will suddenly realize that God is doing for us what we could not do for ourselves.
"""

In [54]:
async def agentic_reflection_pipeline(user_prompt="what is promise 3"):
    # Generator LLM response
    async def generator_response(input_prompt: str) -> AIMessage:
        url = '..........................'
        headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Basic'  
        }
        query = f"[INST] {input_prompt} [/INST] "
        payload = {
            "inputs": "<s>[INST] Prompt [/INST] </s> [INST] Question [/INST] Answer[/INST] </s> " + query,
            "parameters": {
                "max_new_tokens": 4096,
                "temperature": 0.1,
                "repetition_penalty": 1.176,
                "top_k": 20,
                "top_p": 0.1
            }
        }
        payload_json = json.dumps(payload)
        async with httpx.AsyncClient(timeout=60.0) as client:
            response = await client.post(url, headers=headers, data=payload_json)
        return AIMessage(content=response.json()['message'][0]['generated_text'])
        
    async def generation_node(state: Sequence[BaseMessage]):
        input_prompt = ""
        if (len(state) > 1):
            input_prompt += "Question: " + state[0].content + "\n"
            input_prompt += "Generated content:\n" + state[-2].content + "\n"
            input_prompt += "Modify generated content listening to the following suggestions:\n" + state[-1].content
        else: 
            input_prompt += state[0].content
        # input_prompt += "\n".join([msg.content for msg in state[-3:]])
        return await generator_response(input_prompt=input_prompt)

    openai_api_key = "................................"
    client = OpenAI(api_key=openai_api_key)
        
    # Reflection LLM response using OpenAI API
    async def reflect_response(input_prompt: str):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "I will give you a question related to .....: " + document},
                {"role": "user", "content": input_prompt}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        content = response.choices[0].message.content
        return HumanMessage(content=content)
    async def reflection_node(messages: Sequence[BaseMessage]):
        cls_map = {"ai": HumanMessage, "human": AIMessage}
        translated = [messages[0]] + [
            cls_map[msg.type](content=msg.content) for msg in messages[-1:]
        ]
        input_prompt = "\n".join([msg.content for msg in translated])
        res = await reflect_response(input_prompt)
        return HumanMessage(content=res.content)

    builder = MessageGraph()
    builder.add_node("generate", generation_node)
    builder.add_node("reflect", reflection_node)
    builder.set_entry_point("generate")
    
    
    def should_continue(state: List[BaseMessage]):
        if len(state) > 3:
            return END
        return "reflect"
    
    
    builder.add_conditional_edges("generate", should_continue)
    builder.add_edge("reflect", "generate")
    graph = builder.compile()
    
    async def stream_responses():
        final_result = ""
        async for event in graph.astream(
            [
                HumanMessage(
                    content=user_prompt
                )
            ],
        ):
            # print(event)
            # print("---")
            if (list(event.keys())[0] == 'generate'):
                final_result = event['generate'].content
        return final_result

    final_results = await stream_responses()
    print("Final Output:\n" + final_results)
    return final_results

In [52]:
final_output = await agentic_reflection_pipeline(user_prompt="What is promise 1?")

{'generate': AIMessage(content="1st promise: 'No matter how far down the scale we have gone, we will see how our experience can benefit others.' This promise highlights the potential of using personal experiences with addiction to aid in the recovery process of others.", id='0ff77334-413d-4e95-8867-09664068bcfa')}
---
{'reflect': HumanMessage(content='The answer provided for promise 1 is incorrect. Promise 1 is actually: "We are going to know a new freedom and a new happiness." The answer provided deviates from the topic by focusing on the potential of using personal experiences to help others in their recovery process instead of the promise of experiencing new freedom and happiness. It is important to stick to the accurate promises to provide correct information.', id='86cc0487-786c-4121-9c3b-39f44eea08ac')}
---
{'generate': AIMessage(content='\n We are going to know a new freedom and a new happiness. ', id='6be59cf3-92ce-4e99-88a6-2402cf39f4c5')}
---
Final Output:

 We are going to k

In [ ]:
count = 1
for _ in range(12):
    step_query = f"What is promise {count}?"
    final_output = await agentic_reflection_pipeline(user_prompt=step_query)
    count += 1